In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import scipy.signal as signal

In [ ]:
fs = 16000
n_samples = 80000

fs, farend_speech = wavfile.read('D:/Downloads/farend_speech_fileid_0.wav')
fs, nearend_mic_signal = wavfile.read('D:/Downloads/nearend_mic_fileid_0.wav')

farend_speech = farend_speech.astype(np.float32) / 32768.0
nearend_mic_signal = nearend_mic_signal.astype(np.float32) / 32768.0

min_len = min(len(farend_speech), len(nearend_mic_signal))
farend_speech = farend_speech[:min_len]
nearend_mic_signal = nearend_mic_signal[:min_len]

In [ ]:
def nlms_aec(mic_signal, farend_signal, filter_len=64, step_size=0.1):
    n = len(mic_signal)
    w = np.zeros(filter_len)
    output_e = np.zeros(n)   # Tín hiệu sau khi khử vọng
    epsilon = 1e-6 
    
    for i in range(filter_len, n):
        x_vec = farend_signal[i : i - filter_len : -1]
        # 1. Dự đoán Echo
        y = np.dot(w, x_vec)
        # 2. Tính sai số (Tín hiệu sạch ước lượng)
        # e = (Echo + NearSpeech) - EstimatedEcho
        e = mic_signal[i] - y
        output_e[i] = e        
        # 3. Cập nhật trọng số (NLMS Update Rule)
        x_energy = np.dot(x_vec, x_vec) + epsilon
        w = w + (step_size / x_energy) * e * x_vec
        
    return output_e, w

In [ ]:
cleaned_signal, final_weights = nlms_aec(nearend_mic_signal, farend_speech, filter_len=64, step_size=0.5)

In [ ]:
plt.figure(figsize=(14, 8))

# Plot 1: Microphone Signal (Input)
plt.subplot(3, 1, 1)
plt.plot(nearend_mic_signal, color='gray', alpha=0.7, label='Mic Signal (Echo + Speech)')
plt.title("Input: Microphone Signal (Có vùng Double-talk từ 30k-50k)")
plt.legend(loc='upper right')
plt.grid(True)

# Plot 2: Output sau AEC
plt.subplot(3, 1, 2)
plt.plot(cleaned_signal, color='blue', alpha=0.8, label='AEC Output (e)')
plt.title("Output: Tín hiệu sau khi khử vọng (NLMS)")
plt.legend(loc='upper right')
plt.grid(True)

# Plot 3: Zoom vào vùng Double Talk
plt.subplot(3, 1, 3)
zoom_start, zoom_end = 35000, 40000
plt.plot(range(zoom_start, zoom_end), nearend_mic_signal[zoom_start:zoom_end], color='gray', alpha=0.5, label='Mic Input')
plt.plot(range(zoom_start, zoom_end), cleaned_signal[zoom_start:zoom_end], color='blue', label='AEC Output')
plt.title(f"Zoom in Double Talk Region ({zoom_start}-{zoom_end})")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('nlms_real_speech_simulation.png')